# TFT Ranking Model Training

Train a deep learning model to predict TFT match placements using multi-head self-attention.

**Architecture**: Input → FC → 3×Multi-Head Self-Attention (2 heads) → FC → Scores

**Loss**: LambdaNDCG Loss (optimized for ranking)

**Metrics**: NDCG@8, Per-rank Accuracy, MAE, Top-4 Accuracy

## 1. Setup

Install dependencies and upload data file.

In [ ]:
# Install dependencies
!pip install torch torchvision torchaudio
!pip install pytorchltr h5py tensorboard tqdm psutil

In [ ]:
# Upload data file (for Colab)
# Uncomment the following lines if running on Colab

# from google.colab import files
# uploaded = files.upload()  # Upload tft_data_splits.h5

# If running locally or data is already available, just set the path:
DATA_PATH = "tft_data_splits.h5"

In [ ]:
# Clone repository (for Colab)
# Uncomment if running on Colab and need to clone the repo

# !git clone https://github.com/YOUR_USERNAME/TFT_analytics_tool.git
# %cd TFT_analytics_tool

# If running locally, just navigate to the project directory:
import os
# os.chdir('/path/to/TFT_analytics_tool')  # Adjust if needed

## 2. Configuration

Adjust hyperparameters here. All settings are easily configurable.

In [ ]:
from ml.model.config import Config

# ============================================================================
# ADJUST THESE SETTINGS AS NEEDED
# ============================================================================

# Model Architecture
Config.N_ATTENTION_LAYERS = 3      # Number of attention layers
Config.N_HEADS = 2                  # Number of attention heads
Config.HIDDEN_DIM = 256             # Hidden dimension
Config.DROPOUT = 0.2                # Dropout probability

# Training Hyperparameters
Config.BATCH_SIZE = 64              # Batch size
Config.LEARNING_RATE = 1e-4         # Learning rate
Config.EPOCHS = 50                  # Maximum epochs
Config.EARLY_STOPPING_PATIENCE = 10 # Early stopping patience

# Loss Function
Config.LAMBDA_NDCG_SIGMA = 1.0      # LambdaNDCG sigma parameter

# Data
Config.NORMALIZE_FEATURES = True    # Normalize input features
Config.NUM_WORKERS = 2              # DataLoader workers (set to 0 if issues)

# Saving
Config.MODEL_SAVE_DIR = "saved_models"
Config.SAVE_BEST_MODEL = True
Config.SAVE_EVERY_EPOCH = False

# Random Seed
Config.RANDOM_SEED = 42

# ============================================================================

# Print configuration
Config.summary()

## 3. Check GPU Availability

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("Warning: No GPU available. Training will be slow on CPU.")

print(f"\nDevice: {Config.DEVICE}")

## 4. Train Model

This will:
- Load data from HDF5
- Create the model
- Train with LambdaNDCG loss
- Validate after each epoch
- Save the best model
- Log metrics to TensorBoard
- Evaluate on test set

In [ ]:
from ml.model.train import train_model

# Train model
model, test_metrics = train_model(
    hdf5_path=DATA_PATH,
    config=Config,
    verbose=True
)

## 5. View TensorBoard (Optional)

Visualize training progress with TensorBoard.

In [ ]:
# Load TensorBoard extension (for Jupyter/Colab)
%load_ext tensorboard

# Launch TensorBoard
%tensorboard --logdir runs

## 6. Evaluate Saved Model (Optional)

Load and evaluate a previously saved model.

In [ ]:
from ml.model.evaluate import evaluate_saved_model

# Evaluate best model on test set
metrics = evaluate_saved_model(
    model_path="saved_models/best_model.pt",
    hdf5_path=DATA_PATH,
    split="test",
    verbose=True
)

## 7. Make Predictions (Optional)

Use the trained model to make predictions on sample data.

In [ ]:
from ml.model.evaluate import load_model
from ml.model.data_utils import load_and_prepare_data
from ml.model.metrics import scores_to_placements, relevance_to_placements

# Load model
model = load_model("saved_models/best_model.pt", device=Config.DEVICE)

# Load data
data = load_and_prepare_data(DATA_PATH, normalize=True, verbose=False)

# Get a sample
sample_X = data['test_X'][:5]  # First 5 test samples
sample_Y = data['test_Y'][:5]

# Make predictions
model.eval()
with torch.no_grad():
    sample_X = sample_X.to(Config.DEVICE)
    pred_scores = model(sample_X)
    pred_placements = scores_to_placements(pred_scores)

# Convert back to placements
true_placements = relevance_to_placements(sample_Y)

# Print results
print("Sample Predictions:\n")
for i in range(5):
    print(f"Match {i+1}:")
    print(f"  Predicted: {pred_placements[i].tolist()}")
    print(f"  Actual:    {true_placements[i].tolist()}")
    print()

## 8. Download Trained Model (for Colab)

Download the trained model to your local machine.

In [ ]:
# Download best model (for Colab)
# Uncomment if running on Colab

# from google.colab import files
# files.download('saved_models/best_model.pt')

---

## Summary

This notebook trains a TFT ranking model using:
- **Architecture**: Multi-head self-attention (3 layers, 2 heads)
- **Loss**: LambdaNDCG (optimized for ranking)
- **Metrics**: NDCG@8, per-rank accuracy, MAE, Top-4 accuracy

All hyperparameters are easily adjustable in Section 2.

The trained model is saved to `saved_models/best_model.pt`.